In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

In [4]:
ppl = pd.read_csv('../../ddf--entities--person.csv')

In [5]:
ppl

,person,name,state,headquarters,source,industry,image_uri,age,gender,last_name,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb,countries,names
0,a_jayson_adair,A. Jayson Adair,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,usa,A. Jayson Adair
1,a_jerrold_perenchio,A. Jerrold Perenchio,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media;Entertainment,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,NaN,NaN,1931,NaN,NaN,NaN,Chartwell Leisure,1931.0,usa,A. Jerrold Perenchio;Jerrold Perenchio
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,are,Abdul Aziz Al Ghurair
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sau,Abdul Majeed Alhokair
4,abdul_rasyid,Abdul Rasyid,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,idn,Abdul Rasyid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6041,zuo_zongshen,Zuo Zongshen,NaN,NaN,NaN,"Auto Components;Motorcycles, Real Estate;Motor...",NaN,69.0,NaN,NaN,1953-01-01,1953.0,1953,NaN,Male,China-Chongqing,Zongshen,1953.0,chn,Zuo Zongshen;Zuo Zongshen & family
6042,zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,NaN,NaN,NaN,"Motorcycles, Real estate",NaN,NaN,NaN,NaN,NaN,NaN,1953,NaN,NaN,NaN,Zongshen,1953.0,NaN,Zuo Zongshen & Yuan Dexiu
6043,zuowen_song,Zuowen Song,NaN,NaN,"aluminum, diversified",Diversified;Non-ferrous metals processing;Meta...,zuowen-song,75.0,M,Song,1947-03-01,1947.0,1947,NaN,Male,China-Shandong-Yantai,Nanshan,1947.0,chn,Zuowen Song;Song Zuowen;Song Zuowen & Family;S...
6044,zygi_wilf,Zygi Wilf,NaN,NaN,NaN,Sports;Real Estate,NaN,72.0,NaN,NaN,1950-04-22,1950.0,1951,NaN,Male,United States-Millburn,Minnesota Vikings;Garden Homes,1950.0,usa,Zygi Wilf


In [6]:
# TODO: 
# - check entire calculation
# - share over all

In [7]:
synonyms = pd.read_csv('../../../ddf--open_numbers/ddf--synonyms--geo.csv')

In [8]:
syn = synonyms.set_index('synonym')['geo'].to_dict()

In [9]:
# append one that not exists in dict
syn['Eswatini (Swaziland)'] = 'swz'

In [10]:
# def _f(x):
#     try:
#         k = x.split(';')[0]
#         return syn[k]
#     except KeyError:
#         print(x)
#         return x
def _f(x):
    if pd.isnull(x):
        return None
    k = x.split(';')[0]
    if not pd.isnull(k):
        return k
    else:
        return None

In [11]:
# Question: what to do if there are multiple countries?

In [12]:
ppl['geo'] = ppl['countries'].map(_f)

In [13]:
!ls ../../

assets
datapackage.json
datapackage.json.bak
ddf--concepts.csv
ddf--datapoints--annual_income--by--person--time.csv
ddf--datapoints--daily_income--by--person--time.csv
ddf--datapoints--population--by--geo--time--income_group.csv
ddf--datapoints--population_extrapolated--by--geo--time--income_group.csv
ddf--entities--geo--country.csv
ddf--entities--geo.csv
ddf--entities--geo--g77_and_oecd_countries.csv
ddf--entities--geo--global.csv
ddf--entities--geo--income_3groups.csv
ddf--entities--geo--income_groups.csv
ddf--entities--geo--landlocked.csv
ddf--entities--geo--main_religion_2008.csv
ddf--entities--geo--unhcr_region.csv
ddf--entities--geo--unicef_region.csv
ddf--entities--geo--un_sdg_ldc.csv
ddf--entities--geo--un_sdg_region.csv
ddf--entities--geo--west_and_rest.csv
ddf--entities--geo--world_4region.csv
ddf--entities--geo--world_6region.csv
ddf--entities--income_group.csv
ddf--entities--person.csv
etl
README.md
validation-2022-06-21T054605.195Z.log
validation-2022-10-10T034029.182Z.log

In [14]:
all_brackets = np.logspace(-7, 13, 51, endpoint=True, base=2)
brackets_delta = (13 - (-7)) / 50


def bracket_number_from_income(s, monthly=False, integer=True):
    if monthly:  # the input is monthly income, so calculate to daily income first
        res = ((np.log2(s / 30) + 7) / brackets_delta)
    else:
        res = ((np.log2(s) + 7) / brackets_delta)
    if integer:
        return res.astype(int)
    return res

def bracket_number_from_income_c(s):
    res = ((np.log2(s) + 7) / brackets_delta)
    return res.astype(int)

def bracket_number_to_income(n):
    p = n * brackets_delta - 7
    return np.power(2, p)

In [15]:
income = pd.read_csv('../../ddf--datapoints--annual_income--by--person--time.csv')
income2 = pd.read_csv('../../ddf--datapoints--daily_income--by--person--time.csv')

In [16]:
income = income.sort_values(by=['person', 'time'])
income2 = income2.sort_values(by=['person', 'time'])

In [17]:
income.time.unique()

array([2021, 2022, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [18]:
ppl_ = ppl[~pd.isnull(ppl['geo'])]

In [19]:
# 1. convert person to geo NOTE: we will drop names without countries properties
# 2. convert income to income group

person_to_geo = ppl_.set_index('person')['geo'].to_dict()

def _f(x):
    if x in person_to_geo:
        return person_to_geo[x]
    return np.nan

df = income.copy()
df['geo'] = df['person'].map(_f)

df2 = income2.copy()
df2['geo'] = df2['person'].map(_f)

In [20]:
def annual_income_to_group(i):
    daily = i / 365.0
    return bracket_number_from_income(daily)

df['income_group'] = df.apply(lambda x: annual_income_to_group(x['annual_income']), axis=1)

In [21]:
df2['income_group'] = df2['daily_income'].map(bracket_number_from_income)

In [22]:
df2

,person,time,daily_income,geo,income_group
0,a_jayson_adair,2021,72328,usa,57
1,a_jayson_adair,2022,77753,usa,58
2,a_jerrold_perenchio,2002,285715,usa,62
3,a_jerrold_perenchio,2003,241783,usa,62
4,a_jerrold_perenchio,2004,266745,usa,62
...,...,...,...,...,...
32431,zygmunt_solorz_zak,2018,219780,pol,61
32432,zygmunt_solorz_zak,2019,207493,pol,61
32433,zygmunt_solorz_zak,2020,180887,pol,61
32434,zygmunt_solorz_zak,2021,231452,pol,62


In [23]:
res = df.groupby(['geo', 'time', 'income_group'])['person'].count()
res2 = df2.groupby(['geo', 'time', 'income_group'])['person'].count()

In [24]:
res.loc['chn', 2014]

income_group
58    46
59    41
60    49
61    31
62    21
63    16
64     7
65     6
66     5
67     5
68     2
69     2
70     1
Name: person, dtype: int64

In [25]:
_d = res - res2
_d[_d > 0]

Series([], Name: person, dtype: int64)

In [26]:
# reindex to range() and fillna before continue

def reindex_income_group(ser):
    # print(ser)
    ser = ser.set_index('income_group')
    ser = ser.reindex(range(ser.index.min(), ser.index.max()+1)).fillna(0)
    return ser

In [27]:
res = res.reset_index()

In [28]:
res = res.groupby(['geo', 'time']).apply(reindex_income_group)['person']

In [29]:
res = res.sort_index().astype(int)

In [30]:
res.loc['usa', 2022]

income_group
57     25
58    118
59    117
60     95
61    108
62     97
63     83
64     66
65     35
66     22
67     10
68     10
69      4
70      4
71      2
72      7
73      1
74      5
75      1
76      1
77      1
Name: person, dtype: int64

In [31]:
res.loc['ago']

time  income_group
2013  60              1
2014  63              1
2015  62              1
2016  62              1
2017  62              1
2018  61              1
2019  61              1
2020  59              1
Name: person, dtype: int64

In [32]:
df[df.person == 'jeff_bezos']

,person,time,annual_income,geo,income_group
12737,jeff_bezos,2002,60165000,usa,60
12738,jeff_bezos,2003,95925000,usa,62
12739,jeff_bezos,2004,183906000,usa,64
12740,jeff_bezos,2005,142926000,usa,63
12741,jeff_bezos,2006,147318000,usa,64
12742,jeff_bezos,2007,150084000,usa,64
12743,jeff_bezos,2008,273306000,usa,66
12744,jeff_bezos,2009,224808000,usa,65
12745,jeff_bezos,2010,402579000,usa,67
12746,jeff_bezos,2011,579924000,usa,68


In [33]:
ppl[ppl['person'].str.contains('bezos')]

,person,name,state,headquarters,source,industry,image_uri,age,gender,last_name,...,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb,countries,names,geo
2168,jeff_bezos,Jeff Bezos,Washington,WA,Amazon; Amazon.com,Technology;E-Commerce;Online Retail,jeff-bezos,58.0,M,Bezos,...,1964.0,1964,1964,Male,United States-Washington State-Seattle,Amazon,1964.0,usa,Jeff Bezos,usa


In [34]:
res.groupby('time').sum()

time
2002     335
2003     335
2004     435
2005     531
2006     628
2007     760
2008     908
2009     737
2010    1010
2011    1206
2012    1226
2013    1430
2014    1649
2015    1833
2016    1820
2017    2273
2018    2604
2019    2755
2020    2762
2021    3548
2022    3651
Name: person, dtype: int64

In [35]:
res.loc['ago', 2013, :]

income_group
60    1
Name: person, dtype: int64

In [36]:
res.loc['usa', 2022]

income_group
57     25
58    118
59    117
60     95
61    108
62     97
63     83
64     66
65     35
66     22
67     10
68     10
69      4
70      4
71      2
72      7
73      1
74      5
75      1
76      1
77      1
Name: person, dtype: int64

In [37]:
res.name = 'population'

In [38]:
res.to_csv('../../ddf--datapoints--population--by--geo--time--income_group.csv')

In [ ]:
# we are going to use new method. Ignore below methods

In [ ]:
# now get estimation for past and future 

In [ ]:
def get_model(X, Y):
    X = sm.add_constant(X)
    return sm.OLS(Y, X).fit()

def get_predict(model, X):
    a, b = model.params
    Yp = np.array([b * x + a for x in X])
    Yp[Yp < 0] = 0
    return Yp

In [ ]:
res2 = res.reset_index()

In [ ]:
g = res2[(res2['income_group'] == 71) & (res2['geo'] == 'chn')].set_index('time')['person'].copy()

In [ ]:
g

In [ ]:
g = g.reindex(pd.RangeIndex(1998, 2022))

In [ ]:
# g.plot()
X = g.index.values
Y = g.values
yp = get_predict(get_model(X, Y), X)
g2 = g.fillna(pd.Series(yp, index=g.index))

In [ ]:
g2.plot()

In [ ]:
g2

In [ ]:
res2.time.unique()

In [ ]:
def complete_time_series(ser):
    # print(ser)
    if ser.shape[0] < 2:
        print("not enough data point")
        return np.round(ser)
    xmax = ser.index.max()
    xmin = ser.index.min()
    ser = ser.reindex(pd.RangeIndex(xmin, xmax+1))
    #print(ser)
    X = ser.index.values
    Y = ser.fillna(0).values
    xp = np.array(range(xmin-50, xmax+50))
    yp = get_predict(get_model(X, Y), xp)
    
    res = pd.Series(yp, index=xp)
    res.update(ser)
    res.index.name = 'time'
    res.name = 'person'
    res[res < 0] = 0
    return np.round(res)

In [ ]:
res = complete_time_series(res2[res2['income_group'] == 76].set_index('time')['person'])

In [ ]:
res.loc[2000:2010]

In [ ]:
ser = res2[(res2['income_group'] == 70) & (res2['geo'] == 'usa')].set_index('time')['person']

complete_time_series(ser).plot()

In [ ]:
usadf = res2[res2['geo'] == 'usa']

In [ ]:
gs = usadf.groupby('income_group')

def run(df):
    # print(g)
    # print(df)
    return complete_time_series(df[['time', 'person']].set_index('time')['person']).reset_index()
    # res = res.assign(income_group=g).set_index('income_group', append=True)
    # return res

# res = gs.apply(run)

In [ ]:
def run_geo(df):
    # print(df.head(1))
    return  df.groupby('income_group').apply(run).droplevel(1)
    # res.index.names = 

res = dict([(g, run_geo(df)) for g, df in res2.groupby('geo')])

In [ ]:
res_ = []
for k, v in res.items():
    v_ = v.copy()
    v_['geo'] = k
    res_.append(v_)

In [ ]:
res_ = pd.concat(res_)

In [ ]:
res_ = res_.set_index(['geo', 'time'], append=True)

In [ ]:
res_ = res_.reorder_levels([1, 2, 0])

In [ ]:
res_ = res_.sort_index()

In [ ]:
res_.loc['can']

In [ ]:
res_.loc['can', 2018]

In [ ]:
res_.columns = ['population']

In [ ]:
res_.to_csv('../../ddf--datapoints--population--by--geo--time--income_group.csv')

In [ ]:
res_.loc['can']

In [ ]:
res_.loc['can', 2018].plot()

In [ ]:
res_.loc['usa', 2020].plot()

In [ ]:
im = pd.read_csv('../../../ddf--worldbank--povcalnet/etl/datapoints/ddf--datapoints--population--by--geo--time--income_bracket_50.csv')
# im = pd.read_csv('../../../ddf--worldbank--povcalnet/income_mountain/ddf--datapoints--population--by--geo--time--income_bracket_50.csv')

In [ ]:
im = im.set_index(['geo', 'time', 'income_bracket_50'])

In [ ]:
wb = im.loc['usa', 2020].copy()

In [ ]:
fb = res_.loc['usa', 2020].copy()

In [ ]:
fb

In [ ]:
wb.index.name = 'income_group'
fb.index.name = 'income_group'

com = pd.concat([wb, fb])

In [ ]:
com = com.reindex(range(0, 79)).interpolate('slinear')

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]

In [ ]:
wb[46:]['population'].plot(color='r')
fb['population'].plot(color='g')

com[49:58]['population'].astype(int).plot(color='b')

In [ ]:
res_.index.get_level_values('geo').unique()

In [ ]:
import sys
sys.path.insert(0, '../scripts')

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 1

In [ ]:
%aimport smoothlib

In [ ]:
import smoothlib

In [ ]:
from scipy.signal import savgol_filter
import statsmodels.api as sm

In [ ]:
# we don't want to underestimate the millionaires! so, overestimate it!

In [ ]:
!pwd

In [ ]:
def create_shape(wb, fb):
    # remove some wb's tailing datapoint which are lower than
    # the maximum of fb's datapoint
    # and also the head of fb's datapoints which are lower than the maximum
    fb_input = fb['population'].copy()
    wb_input = wb['population'].copy()
    fb_max = fb_input.max()
    for i, v in wb_input.iloc[::-1].iteritems():
        if v > fb_max:
            break        
    wb_input = wb_input.loc[:i]
    idmax = fb_input.idxmax()       
    fb_input = fb_input.loc[idmax:]
    fb_idx_max = fb.index.max()
    
    # inteoplate values. pchip is good at creating curves
    com = pd.concat([wb_input, fb_input])
    com = com.reindex(range(0, fb_idx_max+1))
    com = com.interpolate('pchip')
    
    # smoothing
    begin_idx = i - 5
    ser = com[begin_idx:].copy()
    ser_max = ser.max()
    ser_input = ser / ser_max
    # ser.loc[ser == 0] = 1
    # ser_input = np.log(ser)
    # print(ser)
    # print(ser_input)
    # MAYBE: use log version. But it's not working on first try.
    
    ser_new = smoothlib.run_smooth(ser_input, 8, 0)
    ser_new = smoothlib.run_smooth(ser_new, 5, 0)
    ser_new = smoothlib.run_smooth(ser_new, 3, 0)
    # ser_new = smoothlib.run_smooth(ser_new, 3, 0)
    ser_new = np.round(ser_new * ser_max)
    # print(ser_new)
    # ser_new = np.round(np.exp(ser_new))
    
#     print(begin_idx)
#     print(wb_input.loc[:begin_idx].tail())
#     print(ser_new.head())
    res = pd.concat([wb_input.loc[:begin_idx-1], ser_new])
    return res

In [ ]:
geo, time = ['gbr', 2020]
fb = res_.loc[geo, time].copy()
wb = im.loc[geo, time].copy()

In [ ]:
wb.index.name = 'income_group'
fb.index.name = 'income_group'

In [ ]:
res_shape = create_shape(wb, fb)

In [ ]:
wb[45:]['population'].plot(color='r')
fb['population'].plot(color='g')

# com[49:58]['population'].astype(int).plot(color='b')
res_shape.loc[45:].plot()

In [ ]:
k = 46
wb['population'].loc[k:].plot()
res_shape.loc[k:].plot()

In [ ]:
res_shape.loc[k:]

In [ ]:
im.index.names = ['geo', 'year', 'income_group']

In [ ]:
im

In [ ]:
res_

In [ ]:
im2 = im.reset_index(level=[2])
res_2 = res_.reset_index(level=[2])

In [ ]:
both_idx = im2.index.intersection(res_2.index)

In [ ]:
both_idx

In [ ]:
both_idx[0]

In [ ]:
both_idx.unique()

In [ ]:
# do create shape for all geo time

res = []

for idx in both_idx:
    geo, time = idx
    wb = im.loc[geo, time, :].copy()
    fb = res_.loc[geo, time, :].copy()
    res_shape = create_shape(wb, fb)
    res_shape.name = 'population'
    res_shape = res_shape.to_frame()
    res_shape['geo'] = geo
    res_shape['time'] = time
    res_shape = res_shape.set_index(['geo', 'time'], append=True)
    res.append(res_shape)

In [ ]:
res = pd.concat(res)

In [ ]:
res = res.reorder_levels([1, 2, 0])

In [ ]:
res

In [ ]:
wb_other = im2.loc[~im2.index.isin(both_idx)].set_index('income_group', append=True)

In [ ]:
wb_other.index.names = ['geo', 'time', 'income_group']

In [ ]:
res_all = pd.concat([wb_other, res])

In [ ]:
res.index.intersection(wb_other.index)

In [ ]:
res.index.names

In [ ]:
wb_other.index.names

In [ ]:
res_all = res_all.sort_index()

In [ ]:
res_all['population'] = res_all['population'].astype(int)

In [ ]:
res_all

In [ ]:
res_all.loc['usa', 2020, 46:]

In [ ]:
res_all.loc['usa', 2020, 50:]['population'].plot(color='r')
res_all.loc['usa', 2050, 50:]['population'].plot(color='b')  # room for imporvment?

In [ ]:
res_all.to_csv('../../ddf--datapoints--population_extrapolated--by--geo--time--income_group.csv')

In [ ]:
# create income mountain

In [ ]:
im_all = res_all.unstack()

In [ ]:
im_all = im_all.fillna(0)

In [ ]:
im_all

In [ ]:
res = []

for i, row in im_all.iterrows():
    line = ','.join(row.loc['population'].astype(int).astype(str).values)
    res.append(line)

In [ ]:
res_lines = pd.Series(res, index=im_all.index)

In [ ]:
res_lines.name = 'income_mountain_79bracket_shape_for_log'

In [ ]:
res_lines.index.names = ['country', 'time']

In [ ]:
res_lines

In [ ]:
res_lines.to_csv('../../../ddf--worldbank--povcalnet/income_mountain/ddf--datapoints--income_mountain_79bracket_shape_for_log--by--country--time.csv')

In [ ]:
# DONE!

In [ ]:
fb_max = fb['population'].max()
fb_max

In [ ]:
wb_input = wb['population'].copy()

for i, v in wb_input.iloc[::-1].iteritems():
    if v > fb_max:
        break
        
wb_input = wb_input.loc[:i]

In [ ]:
fb_input = fb['population'].copy()
idmax = fb_input.idxmax()
        
fb_input = fb_input.loc[idmax:]

In [ ]:
fb_input

In [ ]:
fb_idx_max = fb.index.max()

In [ ]:
com = pd.concat([wb_input, fb_input])
com = com.reindex(range(0, fb_idx_max+1))

In [ ]:
com = com.interpolate('pchip')

In [ ]:
# com[40:].plot()
wb['population'][40:].plot()

In [ ]:
ser_input

In [ ]:
smoothlib.get_sample_at_point(42, ser_input, 1, 0, 2)

In [ ]:
smoothlib.mval_at_point(46, ser)

In [ ]:
ser

In [ ]:
45.49 / 94520.000000

In [ ]:
ser = com[40:].copy()
ser_max = ser.max()
ser_input = ser / ser_max
# lowess = sm.nonparametric.lowess(ser.values, ser.index.values, frac=0.22)
# plt.plot(lowess[6:, 0],  lowess[6:, 1])
ser_new = smoothlib.run_smooth(ser_input, 5, 0)
# ser_new = smoothlib.run_smooth(ser_new, 5, 0)

# print(ser_new)
ser_new = ser_new * ser_max
ser_new.index = ser_new.index + 40
# ser_new.plot()
# com[40:].plot()
# fb_input.plot()
# wb_input[46:].plot()

In [ ]:
com[40:]

In [ ]:
ser_new

In [ ]:
# print(ser_new)
# ser_new = ser_new * ser_max
# ser_new.index = ser_new.index + 40
ser_new.loc[46:].plot()
# wb_input.loc[46:].plot(color='r')
# com[46:].plot()

In [ ]:
(np.round(ser_new, 0) - com[40:]).sum()

In [ ]:
com[40:]

In [ ]:
ser.idxmax()

In [ ]:
len(ser)

In [ ]:
wb['population'][46:].sum()

In [ ]:
com[46:].sum()

In [ ]:
lowess

In [ ]:
ser_new

In [ ]:
# below are ways I have tried. not working...

In [ ]:
ser = com[40:]['population'].copy()

In [ ]:
lowess = sm.nonparametric.lowess(ser.values, ser.index.values, frac=0.22)

In [ ]:
plt.plot(lowess[6:, 0],  lowess[6:, 1])

In [ ]:
ser = com['population'].copy()
ser_max = ser.max()
# ser = ser / ser_max
ser_smooth = smoothlib.run_smooth(ser / ser_max, 3, 0)
ser_smooth = smoothlib.run_smooth(ser_smooth, 3, 0)
ser_smooth = smoothlib.run_smooth(ser_smooth, 3, 0)
ser_smooth = ser_smooth * ser_max

ser[46:].plot()
ser_smooth[46:].plot()

In [ ]:
ser.plot()
ser_smooth.plot()

In [ ]:
ser_smooth[48:].plot()

In [ ]:
ser.rolling(8).mean().loc[46:].shift(-8)

In [ ]:
com[40:].plot()

In [ ]:
wb.iloc[-1]

In [ ]:
fb.iloc[1]

In [ ]:
com[49:59]

In [ ]:
# Next: create shapes for all country and save it as datapoint file.
# so that we have population by income group for all 3 datasets.